In [1]:
%run base.ipynb

jit after {'jit': True, 'jit_options': {'flags': '-Ofast'}, 'compiler': 'shell'}
number of joints = 4


In [2]:
i_X_p, i_X_0s = rig_dyn.forward_kinematics(floating_base = True)

In [3]:
H0 , R0, p0 = pluck.spatial_to_homogeneous(i_X_0s[0])
T0 = cs.vertcat(p0, quatT.rotation_matrix_to_quaternion(R0, order='wxyz'))
T0_euler = cs.vertcat(p0, pluck.rotation_matrix_to_euler(R0, order='xyz'))
dIFF_KinJ0 = cs.jacobian(T0_euler, cs.vertcat(base_ss.p_n , arm_ss.q))

H1 , R1, p1 = pluck.spatial_to_homogeneous(i_X_0s[1])
T1 = cs.vertcat(p1, quatT.rotation_matrix_to_quaternion(R1, order='wxyz'))
T1_euler = cs.vertcat(p1, pluck.rotation_matrix_to_euler(R1, order='xyz'))
dIFF_KinJ1 = cs.jacobian(T1_euler, cs.vertcat(base_ss.p_n , arm_ss.q))

H2 , R2, p2 = pluck.spatial_to_homogeneous(i_X_0s[2])
T2 = cs.vertcat(p2, quatT.rotation_matrix_to_quaternion(R2, order='wxyz'))
T2_euler = cs.vertcat(p2, pluck.rotation_matrix_to_euler(R2, order='xyz'))
dIFF_KinJ2 = cs.jacobian(T2_euler, cs.vertcat(base_ss.p_n , arm_ss.q))

H3 , R3, p3 = pluck.spatial_to_homogeneous(i_X_0s[3])
T3 = cs.vertcat(p3, quatT.rotation_matrix_to_quaternion(R3, order='wxyz'))
T3_euler = cs.vertcat(p3, pluck.rotation_matrix_to_euler(R3, order='xyz'))
dIFF_KinJ3 = cs.jacobian(T3_euler, cs.vertcat(base_ss.p_n , arm_ss.q))

H4 , R4, p4 = pluck.spatial_to_homogeneous(i_X_0s[4])
T4 = cs.vertcat(p4, quatT.rotation_matrix_to_quaternion(R4, order='wxyz'))
T4_euler = cs.vertcat(p4, pluck.rotation_matrix_to_euler(R4, order='xyz'))
dIFF_KinJ4 = cs.jacobian(T4_euler, cs.vertcat(base_ss.p_n , arm_ss.q))

fk_eval = cs.Function("fkeval", [arm_ss.q, base_ss.baseT_xyz, base_ss.baseT_rpy, base_ss.p_n], [T0, T1, T2, T3, T4])
fk_eval_euler = cs.Function("fkeval_euler", [arm_ss.q, base_ss.baseT_xyz, base_ss.baseT_rpy, base_ss.p_n], [T0_euler, T1_euler, T2_euler, T3_euler, T4_euler])
dIFF_KinJ = cs.Function("dIFF_KinJ", [arm_ss.q, base_ss.baseT_xyz, base_ss.baseT_rpy, base_ss.p_n], [dIFF_KinJ0, dIFF_KinJ1, dIFF_KinJ2, dIFF_KinJ3, dIFF_KinJ4])
# fk_eval.save("fk_eval.casadi")

In [4]:
# forward kinematics wrt to NED origin quaternion
T_i = fk_eval([cs.pi, cs.pi, cs.pi, cs.pi],alpha.base_T0[3:], alpha.base_T0[:3],[0.2,1,0.5, 0,0,0])
T_i

(DM([0.34, 1.00001, 0.366, 1.24714e-20, -6.12323e-17, 1, 0.000203673]),
 DM([0.36, 1.00002, 0.333, 1, 0.000203673, -6.12448e-17, 6.12199e-17]),
 DM([0.32, 1.00008, 0.1877, -0.000203673, 1, -1.32679e-06, -2.70233e-10]),
 DM([0.34, 1.00009, 0.1547, -0.000176386, 0.866025, 0.500001, 0.000101837]),
 DM([0.34, 1.00013, 0.05495, -0.000176386, 0.866025, 0.500001, 0.000101837]))

In [5]:
fk_eval_euler = cs.Function("fkeval_euler", [arm_ss.q, base_ss.baseT_xyz, base_ss.baseT_rpy, base_ss.p_n], [T4_euler])
fk_eval_euler.save("fk_eval.casadi")
dIFF_KinJ = cs.Function("dIFF_KinJ", [arm_ss.q, base_ss.baseT_xyz, base_ss.baseT_rpy, base_ss.p_n], [dIFF_KinJ4])
dIFF_KinJ.save("dIFF_KinJ.casadi")

In [6]:
# forward kinematics wrt to NED origin euler
T_i = fk_eval_euler([cs.pi, cs.pi, cs.pi, cs.pi],alpha.base_T0[3:], alpha.base_T0[:3],[0.2,1,0.5, 0,0,0])
T_i

DM([0.34, 1.00013, 0.05495, -3.14119, 0, -1.0472])

In [7]:
dKJ = dIFF_KinJ([cs.pi, cs.pi, cs.pi, cs.pi],alpha.base_T0[3:], alpha.base_T0[:3],[0.2,1,0.5, 0,0,0])
dKJ

DM(
[[1, 1.0729e-16, -1.54003e-33, 2.20635e-17, -0.44505, -0.000132355, -5.30718e-08, -0.27805, 0.13275, 8.04935e-17], 
 [7.95347e-17, 1, 2.03288e-20, 0.44505, -1.13028e-17, 0.14, 7.02993e-14, -8.14693e-06, -8.49919e-06, 1.68403e-16], 
 [-1.53999e-33, 2.03288e-20, 1, 0.000132355, -0.14, -1.27372e-18, 2.09251e-17, 0.02, 0.02, -2.99511e-18], 
 [00, 00, 00, 1, -1.75632e-31, 4.31162e-28, -4.31162e-28, -1.22465e-16, -2.65359e-06, 00], 
 [00, 00, 00, 0, -1, -0.000407346, 3.24971e-22, -1, 1, 00], 
 [00, 00, 00, 0, 0.000407346, -1, 1, 3.24971e-22, 1.14413e-33, -1]])

In [7]:
# # c , cpp or matlab code generation for forward dynamics
# fk_eval.generate("fk_eval_.c")
# os.system(f"gcc -fPIC -shared fk_eval_.c -o libFK.so")